In [0]:
%sh
 pip install BeautifulSoup4

Collecting BeautifulSoup4
 Downloading https://files.pythonhosted.org/packages/d1/41/e6495bd7d3781cee623ce23ea6ac73282a373088fcd0ddc809a047b18eae/beautifulsoup4-4.9.3-py3-none-any.whl (115kB)
Collecting soupsieve>1.2; python_version >= "3.0" (from BeautifulSoup4)
 Downloading https://files.pythonhosted.org/packages/6f/8f/457f4a5390eeae1cc3aeab89deb7724c965be841ffca6cfca9197482e470/soupsieve-2.0.1-py3-none-any.whl
Installing collected packages: soupsieve, BeautifulSoup4
Successfully installed BeautifulSoup4-4.9.3 soupsieve-2.0.1
You are using pip version 19.0.3, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [0]:
from pyspark.sql.functions import col, date_format
from pyspark.sql.functions import to_date
from pyspark.sql.types import DateType
WBG_Country = spark.read.csv('/FileStore/tables/HSD_COVID_COUNTRY_MAP.csv',sep = ',',header = True,inferSchema = True)
b= WBG_Country.where((col("COVID_COUNTRY_NAME").isNotNull()) & (col("COVID_COUNTRY_NAME") != ""))
b = b.select(b["WB_COUNTRY_SHORT_NAME"],b["WORLDOMETER_COUNTRY_NAME"])
b = b.dropna()
#b.write.format("delta").saveAsTable("Countries_Temp")
#Country_Selected_1 = Country_Selected.join(b,Country_Selected.location == b.COVID_COUNTRY_NAME,how = 'outer').select(Country_Selected["*"],b["WB_COUNTRY_CODE"],b["WB_COUNTRY_SHORT_NAME"],b["REGION_ABBREVIATION"],b["REGION_NAME"],b["COVID_COUNTRY_RECOVERED_NAME"],b["COVID_COUNTRY_NAME"],b["IBRD_OFFICE_IND"],b["IFC_OFFICE_IND"],b["MIGA_OFFICE_IND"])

#b.write.saveAsTable("HSD_COVID_COUNTRY_MAP")

Code to webscrape Covid data from Worldometer 
Added by Nayan --- 10/21/2020

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pyspark.sql.functions import * 
from datetime import date, timedelta

url = 'https://www.worldometers.info/coronavirus/'
requests.get(url)
page = requests.get(url)

id_irr = ['main_table_countries_today','main_table_countries_yesterday','main_table_countries_yesterday2']
soup = BeautifulSoup(page.text, 'html.parser')
#print(soup)
#table_data = soup.find('table',{'id':'main_table_countries_yesterday'})
for x in id_irr:
  table_data = soup.find('table',{'id':x})
  headers = []
  for i in table_data.find_all('th'):
    title = i.text
    headers.append(title) 
    df = pd.DataFrame(columns = headers)
  for j in table_data.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [tr.text for tr in row_data]
    length = len(df)
    df.loc[length] = row
    df_sp = spark.createDataFrame(df)
    df_sp = df_sp.filter(df_sp['#']!='')
    df_sp = df_sp.withColumnRenamed("Country,Other","Country_1")
  if x == 'main_table_countries_today':
    today = date.today()
    table_countries_today = df_sp
    table_countries_today = table_countries_today.select("Country_1","TotalTests")
    table_countries_today = table_countries_today.withColumn("Date", lit(today.strftime("%m-%d-%Y")))
  elif x == 'main_table_countries_yesterday':
    table_countries_yesterday = df_sp
    yesterday = today - timedelta(days = 1)
    table_countries_yesterday = table_countries_yesterday.select("Country_1","TotalTests")
    table_countries_yesterday = table_countries_yesterday.withColumn("Date", lit(yesterday.strftime("%m-%d-%Y")))
  elif x == 'main_table_countries_yesterday2':
    yesterday2 = today - timedelta(days = 2)
    table_countries_yesterday2 = df_sp
    table_countries_yesterday2 = table_countries_yesterday2.select("Country_1","TotalTests")
    table_countries_yesterday2 = table_countries_yesterday2.withColumn("Date", lit(yesterday2.strftime("%m-%d-%Y")))
  ################################################## Convert to spark Dataframe####################  
  
final_Countries_tests_df = table_countries_today.unionAll(table_countries_yesterday.unionAll(table_countries_yesterday2))

final_Countries_tests_df = final_Countries_tests_df.join(b,final_Countries_tests_df.Country_1 == b.WORLDOMETER_COUNTRY_NAME,how = 'inner').select(final_Countries_tests_df["Date"],final_Countries_tests_df["TotalTests"],b["WB_COUNTRY_SHORT_NAME"])
final_Countries_tests_df = final_Countries_tests_df.withColumnRenamed("WB_COUNTRY_SHORT_NAME","Country")
final_Countries_tests_df = final_Countries_tests_df.filter(final_Countries_tests_df.Country.isNotNull())


#final_Countries_tests_df.write.format("delta").partitionBy("Date").saveAsTable("Countries_Tests_1")

In [0]:
%sql
DROP TABLE IF EXISTS Countries_Tests_Stage

In [0]:
final_Countries_tests_df.write.format("delta").partitionBy("Date").saveAsTable("Countries_Tests_Stage")

In [0]:
%sql
MERGE INTO Countries_Tests
USING Countries_Tests_Stage
ON Countries_Tests.Date = Countries_Tests_Stage.Date AND Countries_Tests.Country = Countries_Tests_Stage.Country
WHEN MATCHED THEN
  UPDATE SET Countries_Tests.TotalTests = Countries_Tests_Stage.TotalTests
WHEN NOT MATCHED
  THEN INSERT (Country,TotalTests , Date) VALUES (Country,TotalTests , Date)

In [0]:
%sql
SELECT *  FROM Countries_Tests Order By Date

Country,TotalTests,Date
Angola,85213,09-28-2020
Brazil,17900000,09-28-2020
Dominican Republic,472421,09-28-2020
Liechtenstein,900,09-28-2020
Lithuania,761616,09-28-2020
"Congo, Democratic Republic of",null,09-28-2020
Luxembourg,818380,09-28-2020
Madagascar,70281,09-28-2020
Ecuador,419740,09-28-2020
Malawi,52790,09-28-2020
